In [ ]:
# Initialize Otter
import otter
grader = otter.Notebook("hw04.ipynb")

# Homework 4: Wrangling the Vote 
(Cleaning, Outliers, and Missing Values) 

<!-- BEGIN QUESTION -->

**Question:** What is your name?

_Type your answer here, replacing this text._

<!-- END QUESTION -->

Please complete this notebook by filling in the cells provided. 

**Throughout this homework and all future ones, please be sure to not re-assign variables throughout the notebook!** For example, if you use `max_temperature` in your answer to one question, do not reassign it later on. Moreover, please be sure to only put your written answers in the provided cells. 

In [ ]:
# Don't change this cell; just run it. 

import numpy as np
from datascience import *

## Voting in Athens County, Ohio

The Ohio Secretary of State provides csv files of registered voters on its page
[Statewide Voter Files download, by county](https://www6.ohiosos.gov/ords/f?p=111:1).
There is also a button where you can download a description of the Voter File Layout. 

Below is the data from Athens County, downloaded on January 27, 2023. 

In [ ]:
# just run this cell
# (To make things run faster we skip columns about voting prior to 2022.)
registered = Table().read_table("Athens_voter_registered_20230127.csv").select(*list(range(46)),114,115,116)
registered.show(4)

The Athens County Board of Elections provides [csv files for those who voted in specific elections](https://lookup.boe.ohio.gov/vtrapp/athens/vtrreport.aspx). 

The following file was downloaded on January 30, 2023, using the options
* Downloadable File (Comma Separated Values)
* 11/8/2022 General Election, 8/2/2022 Primary Election, 5/3/2022 Primary Election
* OR - Meaning any elections could meet the criteria
* All Current Part Affiliations
* No Filtering
* Alphabetical
* Yes, Mailing Addresses in addition to Residential Addresses?

(There were two primaries in 2022 due to court battle over redistricting.)

In [ ]:
# just run this cell
# (To make things run faster we skip columns about voting prior to 2022.)
voted = Table().read_table("Athens_voter_voted_20230130.csv").select(*list(range(33))) 
voted.show(3)

## 1. Registration versus voting

The `registered` table has a column `SOS_VOTERID`, which the documentation says is
> The Secretary of State centralized system’s generated unique number, assigned to each voter registration record.

The `voted` table has this same number in the column `SOSIDNUM`.

One expects that some people who are registered did not vote.
Did any vote who are not registered?

The python `set` object has a method [`difference`](https://docs.python.org/3/library/stdtypes.html#frozenset.difference) that will help. 
It gives the set of elements in the initial set that are not in the other set(s), as illustrated below.

In [ ]:
set(['a','b','c','d']).difference(set(['a','c','e']))

**Question 1.1:** Assign `ID_voted_not_registered` to the set of all IDs for people who voted but are not registered.

In [ ]:
ID_voted_not_registered = ...
ID_voted_not_registered

In [ ]:
grader.check("q1_1")

**Question 1.2:** Assign `voted_not_registered` to a table with the rows from `voted` of people who are not registered.

(Hint: Use [`tbl.where`](http://www.data8.org/datascience/reference-nb/datascience-reference.html#tbl.where()) with [`are.contained_in`](http://www.data8.org/datascience/reference-nb/datascience-reference.html#are.contained_in()).)

In [ ]:
voted_not_registered = ...
voted_not_registered

In [ ]:
grader.check("q1_2")

<!-- BEGIN QUESTION -->

**Question 1.3:** Why might there be people who voted but are not registered?

_Type your answer here, replacing this text._

<!-- END QUESTION -->

## 2. Voter birthdays and years

The `DATE_OF_BIRTH` column in `registered` has the date of birth. 

**Question 2.1:** Sort `registered` so the earliest births are first, then set `earliest_registered_birth` to the earliest date.

In [ ]:
registered_sorted_by_birth = ...
registered_sorted_by_birth.show(5)
earliest_registered_birth = ...
earliest_registered_birth

In [ ]:
grader.check("q2_1")

The `BIRTHYEAR` column in `voted` has the year of birth.

**Question 2.2:** Sort `voted` so the earliest births are first, then set `earliest_voted_birth` to the earliest year.

In [ ]:
voted_sorted_by_birth = ...
voted_sorted_by_birth.show(5)
earliest_voted_birth = ...
earliest_voted_birth

In [ ]:
grader.check("q2_2")

<!-- BEGIN QUESTION -->

**Question 2.3:** Are these earliest birthdays and years plausible?
Are they outliers or errors?

_Type your answer here, replacing this text._

<!-- END QUESTION -->

**Question 2.4:** Sort `registered` so the latest births are first, then set `latest_registered_birth` to the latest date.

In [ ]:
registered_sorted_by_birth_descending = ...
registered_sorted_by_birth_descending.show(5)
latest_registered_birth = ...
latest_registered_birth

In [ ]:
grader.check("q2_4")

**Question 2.5:** Sort `voted` so the latest births are first, then set `latest_voted_birth` to the latest year.

In [ ]:
voted_sorted_by_birth_descending = ...
voted_sorted_by_birth_descending.show(5)
latest_voted_birth = ...
latest_voted_birth

In [ ]:
grader.check("q2_5")

<!-- BEGIN QUESTION -->

**Question 2.6:** 
Voters are supposed to be at least 18 years old on the date of the general election. 
Are these latest birthdays and years consistent with that requirement? 
If not, then what might be the cause?

_Type your answer here, replacing this text._

<!-- END QUESTION -->

We can find out how many voters were born each year using `group`, and then sort by year or by count.

In [ ]:
voted.group('BIRTHYEAR').sort('BIRTHYEAR')

In [ ]:
voted.group('BIRTHYEAR').sort('count',descending=True)

However, there are too many years involved to get a good understanding of the distribution.
To simplify, we can first bin voters into their decade, and then group by decade.

Here is function that converts a year to a decade.
It uses the python `//` operation, which means divide integers and discard the remainder

In [ ]:
def year_to_decade(year):
    return (year//10)*10
# simple test
year_to_decade(1949)

**Question 2.7** Assign `voted_with_decade` to a `Table` consisting of `voted` and an additional column `BIRTHDECADE` with the decade of birth.

In [ ]:
voted_with_decade = ...
voted_with_decade.show(3)
# inspect the decade column
voted_with_decade.select('BIRTHYEAR','BIRTHDECADE')

In [ ]:
grader.check("q2_7")

Now we can group to see the distribution:

In [ ]:
voted_with_decade.group('BIRTHDECADE').sort('BIRTHDECADE').show(11)

## 3. Party Affiliation

We can also use `group` to check party affiliations, in the column `PARTYAFFIL`.

In [ ]:
voted.group('PARTYAFFIL')

Since there are so few of them, let's consider those with affiliation `IND` or `LIB` to be outliers.
They are not errors, but would clutter our analysis.

Looking back out our `BIRTHDECADE` information, the decades `1900` and `1910` are also outliers, which would clutter our analysis.

**Question 3.1:** Starting from `voted_with_decade`, create `voted_with_decade_noout` than has removed rows with affiliation `IND` or `LIB` or `BIRTHDECADE` `1900` or `1910`.

In [ ]:
voted_with_decade_noout = ...

# check
voted_with_decade_noout.group('PARTYAFFIL').show()
voted_with_decade_noout.group('BIRTHDECADE').show(3)
voted_with_decade_noout.show(3)

In [ ]:
grader.check("q3_1")

The `pivot` method can show counts of combinations of entries in two columns.
For example here is the pivot table of `PARTYAFFIL` versus `SUFFIXN`.

In [ ]:
voted_with_decade_noout.pivot('PARTYAFFIL','SUFFIXN')

**Question 3.2:** From `voted_with_decade_noout`, make the pivot table of `PARTYAFFIL` versus `BIRTHDECADE`.

In [ ]:
voted_party_vs_decade = ...
voted_party_vs_decade

In [ ]:
grader.check("q3_2")

The way in which a voter voted in the 2022 general election is in the column `20221108G_TYPE`.

In [ ]:
voted_with_decade_noout.group('20221108G_TYPE')

<!-- BEGIN QUESTION -->

**Question 3.3:** Which party affiliation(s) prefered to use absentee ballots?

_Type your answer here, replacing this text._

In [ ]:
# Run some code here, then write your answer in the text cell, explaining how you came to that conclusion
...

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

**Question 3.4:** One might think that older voters would use absentee ballots more than younger voters.
Does this data support that conclusion?

_Type your answer here, replacing this text._

In [ ]:
# Run some code here, then write your answer in the text cell, explaining how you came to that conclusion
...

<!-- END QUESTION -->

## 4.  Checking consistency

As noted above, the `SOSIDNUM` column in `voted` has the same ID number as the `SOS_VOTERID` in `registered`.
We can use this ID number and the [`join` method](http://www.data8.org/datascience/reference-nb/datascience-reference.html#tbl.join()) to combine the two tables.

(Note that when we join tables this way, only rows that have a match between the two tables are retained.)



**Question 4.1** Join `registered` onto `voted`.

(Do it in the direction `voted.join(...)` rather than `registered.join(...)`.)

In [ ]:
voted_joined = ...
voted_joined.show(3)

In [ ]:
grader.check("q4_1")

**Question 4.2:** Construct the pivot table of `PARTYAFFIL` versus `PARTY_AFFILIATION`.

In [ ]:
affiliation_pivot = ...
affiliation_pivot

In [ ]:
grader.check("q4_2")

<!-- BEGIN QUESTION -->

**Question 4.3:** How well does the party affiliation information match?

Do you observe any patterns in the mismatch?

_Type your answer here, replacing this text._

<!-- END QUESTION -->

One would hope that the year in the `DATE_OF_BIRTH` matches the year in `BIRTHYEAR`. 
Let's find all the cases when it does not.

The contents of the `DATE_OF_BIRTH` column are strings with format like `'1919-03-24'`, so we need to
1. Extract the first 4 characters (to get `'1919'`).
2. Convert that to an int (to get `1919`)

Selecting the first part of a string works like this example.

In [ ]:
'Sunday'[:3]

**Question 4.4:** Construct a function that converts a date in `'yyyy-mm-dd'` format to the integer `yyyy`.

In [ ]:
def yearnumber(x):
    ...
# simple test
yearnumber('1919-03-24')

In [ ]:
grader.check("q4_4")

**Question 4.5:** Assign `difference_in_years` to an array with the year from `registered` minus the year from `voted`.

In [ ]:
# You could do this in one line
year_in_DATE_OF_BIRTH = ...
difference_in_years = ...
difference_in_years

In [ ]:
grader.check("q4_5")

**Question 4.6:** Assign `voted_year_errors` to a table with the rows from `voted_joined` 
where the year in the `DATE_OF_BIRTH` column does not match the year in `BIRTHYEAR`.

(Hint: Use `difference_in_years` that you created above.)

In [ ]:
# You could do this in one long line
voted_joined_with_difference = ...
voted_year_errors = ...
voted_year_errors

In [ ]:
grader.check("q4_6")

## Feedback

<!-- BEGIN QUESTION -->

**Feedback?:** This assignment is new. 
* Do you have any feedback or suggestions on how to improve it?
* Are there any questions that have unclear instructions?
* Is there anything else that you think would be interesting to do with this data?

_Type your answer here, replacing this text._

<!-- END QUESTION -->

## Acknowledgements

<!-- BEGIN QUESTION -->

**Acknowledgements:**
If you received help from any person other than your instructor, then describe here what help you received and from whom. 
If not, then say 'None'.

_Type your answer here, replacing this text._

<!-- END QUESTION -->

It is forbidden to: 
* Use the work or programs from students who took this class (in any of its versions at any university).
* Use websites that claim to have solutions for this class (in any of its versions at any university).
* Directly copy from a classmate.

**Did you do anything forbidden while working on this assignment?**

Assign `did_forbidden` to either `True` or `False`.

In [ ]:
did_forbidden = ...

In [ ]:
grader.check("q_forbidden")

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

After exporting, submit the .zip file in BlackBoard

In [ ]:
# Save your notebook first, then run this cell to export your submission.
grader.export(run_tests=True)